In [49]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

from train import LogTransformer
import pickle

In [50]:
train_path = "../data/complex_data.csv"
data = pd.read_csv(train_path, sep='\t', index_col=0)
data.head()

,Log price,Log area (square meters),Distance to center 2,Distance to center 4,Log WC,Log Floors
0,-0.530310,-1.288856,0.172328,1.388260,0.323821,1.033543
1,1.518783,-0.637139,-4.732391,-0.330580,-1.312694,-0.679309
2,-0.767807,0.330336,0.192896,-0.764218,0.323821,-0.679309
3,-0.130805,-1.349877,0.279746,-0.718391,-1.312694,-0.876262
4,-0.099847,-0.362392,0.316293,-0.713225,-1.312694,-0.679309


In [51]:
data.describe()

,Log price,Log area (square meters),Distance to center 2,Distance to center 4,Log WC,Log Floors
count,1.548000e+03,1.548000e+03,1.548000e+03,1.548000e+03,1.548000e+03,1.548000e+03
mean,2.088482e-16,-3.029446e-16,-5.519558e-16,1.204893e-17,1.067191e-16,2.891744e-16
std,1.000323e+00,1.000323e+00,1.000323e+00,1.000323e+00,1.000323e+00,1.000323e+00
min,-2.639511e+00,-5.099559e+00,-4.800193e+00,-1.487133e+00,-2.949209e+00,-1.975031e+00
25%,-6.644242e-01,-7.237111e-01,1.315263e-01,-7.301941e-01,-3.553939e-01,-6.793085e-01
50%,-1.279648e-01,-5.994834e-02,2.279517e-01,-7.101910e-01,3.238213e-01,7.864041e-02
75%,5.513539e-01,5.662177e-01,2.989341e-01,1.370351e+00,3.238213e-01,6.164137e-01
max,2.759917e+00,3.689734e+00,2.169470e+00,1.997407e+00,6.190667e+00,4.253966e+00


In [52]:
X = data.drop(columns=["Log price"])
y = data["Log price"]

In [53]:
k_number = int(data.shape[0]/500)

# Support Vector Machine Regressor

In [54]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import expon, randint

svr = SVR()

param_distribution = [
    {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100], 'kernel': ['rbf']}, 
    {'C': [0.1, 1, 10, 100], 'degree': [0.1, 1, 10, 100], 'coef0': [1, 2, 3, 4], 'kernel': ['poly']},
    {'C': [0.1, 1, 10, 100], 'kernel': ['linear']}, 
    {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100], 'coef0': [1, 2, 3, 4], 'kernel': ['sigmoid']},
]

random_search = RandomizedSearchCV(
    svr, 
    param_distributions=param_distribution, 
    n_iter=100, 
    scoring="neg_root_mean_squared_error",
    cv=k_number, 
    random_state=1, 
    n_jobs=-1
)

random_search.fit(X, y)

In [ ]:
cv_res = pd.DataFrame(random_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

cv_res = cv_res[
    [
        'param_kernel', 
        'param_degree', 
        'param_coef0', 
        'param_C',
        'split0_test_score',
        'split1_test_score', 
        'split2_test_score',
        'mean_test_score'
    ]
]
score_cols = ["split0", "split1", "split2", "mean_test"]
cv_res.columns = ["kernel", "degree", "coef0", "C"] + score_cols
cv_res[score_cols] = np.sqrt(-cv_res[score_cols])

cv_res.head()

,kernel,degree,coef0,C,split0,split1,split2,mean_test
0,poly,1,2,1,0.904775,0.896291,0.875969,0.892427


In [ ]:
cv_res.columns

Index(['kernel', 'degree', 'coef0', 'C', 'split0', 'split1', 'split2',
       'mean_test'],
      dtype='object')